In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, GRU
from keras.callbacks import EarlyStopping





In [ ]:
data = pd.read_csv('weather_prediction_dataset.csv')

In [ ]:
data

,DATE,MONTH,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,STOCKHOLM_temp_min,STOCKHOLM_temp_max,TOURS_wind_speed,TOURS_humidity,TOURS_pressure,TOURS_global_radiation,TOURS_precipitation,TOURS_temp_mean,TOURS_temp_min,TOURS_temp_max
0,20000101,1,8,0.89,1.0286,0.20,0.03,0.0,2.9,1.6,...,-9.3,0.7,1.6,0.97,1.0275,0.25,0.04,8.5,7.2,9.8
1,20000102,1,8,0.87,1.0318,0.25,0.00,0.0,3.6,2.7,...,0.5,2.0,2.0,0.99,1.0293,0.17,0.16,7.9,6.6,9.2
2,20000103,1,5,0.81,1.0314,0.50,0.00,3.7,2.2,0.1,...,-1.0,2.8,3.4,0.91,1.0267,0.27,0.00,8.1,6.6,9.6
3,20000104,1,7,0.79,1.0262,0.63,0.35,6.9,3.9,0.5,...,2.5,4.6,4.9,0.95,1.0222,0.11,0.44,8.6,6.4,10.8
4,20000105,1,5,0.90,1.0246,0.51,0.07,3.7,6.0,3.8,...,-1.8,2.9,3.6,0.95,1.0209,0.39,0.04,8.0,6.4,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3649,20091228,12,7,0.82,1.0084,0.28,0.42,0.3,3.2,1.0,...,-2.7,2.4,3.7,0.95,1.0011,0.22,1.50,6.2,1.8,10.6
3650,20091229,12,7,0.92,1.0028,0.22,1.68,0.2,4.5,2.4,...,-9.5,0.8,5.3,0.89,0.9966,0.24,0.40,10.4,6.2,14.5
3651,20091230,12,8,0.92,0.9979,0.07,1.54,0.0,8.5,7.5,...,-12.5,-7.4,3.8,0.88,0.9939,0.24,1.00,10.0,8.7,11.3
3652,20091231,12,7,0.93,0.9958,0.17,0.57,0.1,6.6,4.3,...,-9.3,-6.5,4.2,0.88,0.9933,0.58,0.02,8.5,6.2,10.9


In [ ]:
features = ['DATE', 'TOURS_temp_max', 'TOURS_temp_min', 'TOURS_temp_mean']
data = data[features]

In [ ]:
data.loc[:, 'DATE'] = pd.to_datetime(data.loc[:, 'DATE'])


<ipython-input-63-e88ac2bc207b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'DATE'] = pd.to_datetime(data.loc[:, 'DATE'])


In [ ]:
data

,DATE,TOURS_temp_max,TOURS_temp_min,TOURS_temp_mean
0,1970-01-01 00:00:00.020000101,9.8,7.2,8.5
1,1970-01-01 00:00:00.020000102,9.2,6.6,7.9
2,1970-01-01 00:00:00.020000103,9.6,6.6,8.1
3,1970-01-01 00:00:00.020000104,10.8,6.4,8.6
4,1970-01-01 00:00:00.020000105,9.5,6.4,8.0
...,...,...,...,...
3649,1970-01-01 00:00:00.020091228,10.6,1.8,6.2
3650,1970-01-01 00:00:00.020091229,14.5,6.2,10.4
3651,1970-01-01 00:00:00.020091230,11.3,8.7,10.0
3652,1970-01-01 00:00:00.020091231,10.9,6.2,8.5


In [ ]:
train_set = data[(data['DATE'] >= '1970-01-01 00:00:00.020000101') & (data['DATE'] < '1970-01-01 00:00:00.020080101')]
val_set = data[(data['DATE'] >= '1970-01-01 00:00:00.020080101') & (data['DATE'] < '1970-01-01 00:00:00.020100101')]


In [ ]:
features = ['TOURS_temp_min'	,'TOURS_temp_mean']
target = 'TOURS_temp_max'


In [ ]:
X_train = train_set[features].values
y_train = train_set[target].values

X_val = val_set[features].values
y_val = val_set[target].values


In [ ]:
X_train.shape

(2922, 2)

In [ ]:
train_set

,DATE,TOURS_temp_max,TOURS_temp_min,TOURS_temp_mean
0,1970-01-01 00:00:00.020000101,9.8,7.2,8.5
1,1970-01-01 00:00:00.020000102,9.2,6.6,7.9
2,1970-01-01 00:00:00.020000103,9.6,6.6,8.1
3,1970-01-01 00:00:00.020000104,10.8,6.4,8.6
4,1970-01-01 00:00:00.020000105,9.5,6.4,8.0
...,...,...,...,...
2917,1970-01-01 00:00:00.020071227,8.8,3.7,6.2
2918,1970-01-01 00:00:00.020071228,6.8,3.5,5.2
2919,1970-01-01 00:00:00.020071229,9.3,2.9,6.1
2920,1970-01-01 00:00:00.020071230,4.5,1.1,2.8


In [ ]:
val_set

,DATE,TOURS_temp_max,TOURS_temp_min,TOURS_temp_mean
2922,1970-01-01 00:00:00.020080101,4.5,0.9,2.7
2923,1970-01-01 00:00:00.020080102,3.8,-1.1,1.3
2924,1970-01-01 00:00:00.020080103,7.1,0.2,3.6
2925,1970-01-01 00:00:00.020080104,10.0,6.0,8.0
2926,1970-01-01 00:00:00.020080105,10.9,8.0,9.4
...,...,...,...,...
3648,1970-01-01 00:00:00.020091227,9.7,-0.5,4.6
3649,1970-01-01 00:00:00.020091228,10.6,1.8,6.2
3650,1970-01-01 00:00:00.020091229,14.5,6.2,10.4
3651,1970-01-01 00:00:00.020091230,11.3,8.7,10.0


In [ ]:
# Assuming your X_train and X_test data have shape (num_samples, num_features)
timesteps = 1
X_train = X_train.reshape(-1, timesteps, X_train.shape[1])
y_train = y_train.reshape(-1, timesteps,1)
X_val = X_val.reshape(-1, timesteps, X_val.shape[1])
y_val = y_val.reshape(-1, timesteps,1)

#X_test = X_test.reshape(-1, timesteps, input_dim)

In [ ]:
y_val.shape

(731, 1, 1)

In [ ]:
# Define the architecture for each model
rnn_model = Sequential()
rnn_model.add(SimpleRNN(units=32, activation='tanh', input_shape=(1, 2)))
rnn_model.add(Dense(units=1))
rnn_model.compile(optimizer='adam', loss='mean_squared_error')

gru_model = Sequential()
gru_model.add(GRU(units=32, activation='tanh', input_shape=(1, 2)))
gru_model.add(Dense(units=1))
gru_model.compile(optimizer='adam', loss='mean_squared_error')

lstm_model = Sequential()
lstm_model.add(LSTM(units=32, activation='tanh', input_shape=(1, 2)))
lstm_model.add(Dense(units=1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

# Train each model with early stopping
es_callback = EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose=1)

rnn_history = rnn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, callbacks=[es_callback])
gru_history = gru_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, callbacks=[es_callback])
lstm_history = lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, callbacks=[es_callback])

# Try three different step sizes for each model and compare their loss
step_sizes = [0.1, 1, 10]

for step_size in step_sizes:
    X_train_step = np.cumsum(X_train, axis=1) * step_size
    X_val_step = np.cumsum(X_val, axis=1) * step_size
    
    rnn_step_history = rnn_model.fit(X_train_step, y_train, validation_data=(X_val_step, y_val), epochs=100, batch_size=32, callbacks=[es_callback])
    gru_step_history = gru_model.fit(X_train_step, y_train, validation_data=(X_val_step, y_val), epochs=100, batch_size=32, callbacks=[es_callback])
    lstm_step_history = lstm_model.fit(X_train_step, y_train, validation_data=(X_val_step, y_val), epochs=100, batch_size=32, callbacks=[es_callback])
    
    print(f'Step size: {step_size}')
    print(f'RNN validation loss: {rnn_step_history.history["val_loss"][-1]}')
    print(f'GRU validation loss: {gru_step_history.history["val_loss"][-1]}')
    print(f'LSTM validation loss: {lstm_step_history.history["val_loss"][-1]}')
    

Epoch 1/100
92/92 [==============================] - 2s 6ms/step - loss: 322.3556 - val_loss: 231.8502
Epoch 2/100
92/92 [==============================] - 0s 3ms/step - loss: 182.9454 - val_loss: 134.0833
Epoch 3/100
92/92 [==============================] - 0s 3ms/step - loss: 117.0878 - val_loss: 85.3240
Epoch 4/100
92/92 [==============================] - 0s 3ms/step - loss: 78.9667 - val_loss: 62.2649
Epoch 5/100
92/92 [==============================] - 0s 3ms/step - loss: 59.9495 - val_loss: 47.3276
Epoch 6/100
92/92 [==============================] - 0s 3ms/step - loss: 47.2780 - val_loss: 37.3459
Epoch 7/100
92/92 [==============================] - 0s 3ms/step - loss: 38.4224 - val_loss: 30.2502
Epoch 8/100
92/92 [==============================] - 0s 3ms/step - loss: 31.7337 - val_loss: 24.7597
Epoch 9/100
92/92 [==============================] - 0s 3ms/step - loss: 26.2862 - val_loss: 20.0695
Epoch 10/100
92/92 [==============================] - 0s 3ms/step - loss: 21.7313 - va